In [ ]:
!pip install transformers datasets sacrebleu sentencepiece accelerate

In [ ]:
from transformers import MarianMTModel, MarianTokenizer

model_name = "Helsinki-NLP/opus-mt-ar-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
# !rm -rf ~/.cache/huggingface/datasets/MBZUAI-Paris_DODa-10K
# !git clone https://huggingface.co/datasets/MBZUAI-Paris/DODa-10K
!git clone https://huggingface.co/datasets/midox05/train_ready_english_darija_dataset

Cloning into 'train_ready_english_darija_dataset'...
remote: Enumerating objects: 7, done.
remote: Total 7 (delta 0), reused 0 (delta 0), pack-reused 7 (from 1)
Unpacking objects: 100% (7/7), 2.25 KiB | 329.00 KiB/s, done.


In [ ]:
from datasets import load_dataset, Dataset
import pandas as pd
import random

# dataset = load_dataset('midox05/train_ready_english_darija_dataset', split='train')
dataset = pd.read_csv("/content/train_ready_english_darija_dataset/ready_df.csv")


In [ ]:
dataset = Dataset.from_pandas(dataset)
random_indices = random.sample(range(len(dataset)), 80000)
random_dataset = dataset.select(random_indices)
dataset = random_dataset.train_test_split(test_size=0.1)

In [ ]:
def preprocess(examples):
    inputs = examples["darija"]
    targets = examples["english"]
    model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=128)
    labels = tokenizer(targets, padding="max_length", truncation=True, max_length=128)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized = dataset.map(preprocess, batched=True)


Map:   0%|          | 0/45000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_metric
import numpy as np

metric = load_metric("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = metric.compute(predictions=decoded_preds, references=[[l] for l in decoded_labels])
    return {"bleu": result["score"]}


<ipython-input-7-c6454bb381e0>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./mt_ar_en_finetuned",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    num_train_epochs=2,
    weight_decay=0.01,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=10,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

<ipython-input-25-256bdafe26ac>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu
1,0.500500,0.464734,48.004383
2,0.410800,0.442144,49.541535


TrainOutput(global_step=11250, training_loss=0.3634017863167657, metrics={'train_runtime': 4284.1743, 'train_samples_per_second': 21.008, 'train_steps_per_second': 2.626, 'total_flos': 3050852843520000.0, 'train_loss': 0.3634017863167657, 'epoch': 2.0})

In [ ]:
import torch

def translate(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(model.device)
    with torch.no_grad():
        translated = model.generate(**inputs, max_length=128)
    return tokenizer.batch_decode(translated, skip_special_tokens=True)


In [ ]:
sentences = [
    "فين مشيتي البارح؟",
    "بغيت نشرب قهوة، كتجي؟",
    "التيليفون ديالي سالا من الشارج",
    "راك درتي مزيان ف الخدمة",
    "خصني نمشي بكري اليوم"
]

translations = translate(sentences)

for en, ar in zip(sentences, translations):
    print(f"EN: {en}\nAR: {ar}\n")


EN: فين مشيتي البارح؟
AR: where did you go yesterday

EN: بغيت نشرب قهوة، كتجي؟
AR: I want to drink coffee, you come by?

EN: التيليفون ديالي سالا من الشارج
AR: My phone salla from the charge

EN: راك درتي مزيان ف الخدمة
AR: You did great at work

EN: خصني نمشي بكري اليوم
AR: I need to go early today



In [ ]:
model.save_pretrained("eng-darija-trsl")
tokenizer.save_pretrained("eng-darija-trsl")

('eng-darija-trsl/tokenizer_config.json',
 'eng-darija-trsl/special_tokens_map.json',
 'eng-darija-trsl/vocab.json',
 'eng-darija-trsl/source.spm',
 'eng-darija-trsl/target.spm',
 'eng-darija-trsl/added_tokens.json')

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `ProjetDl` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `ProjetDl`

In [ ]:
# Replace with your model repo name
model_name_on_hub = "nwiser/eng-darija-trsl"

# Push model
model.push_to_hub(model_name_on_hub)

# Push tokenizer
tokenizer.push_to_hub(model_name_on_hub)

model.safetensors:   0%|          | 0.00/306M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nwiser/eng-darija-trsl/commit/41e1a119e00684e88d539b35190194b7a2c825df', commit_message='Upload tokenizer', commit_description='', oid='41e1a119e00684e88d539b35190194b7a2c825df', pr_url=None, repo_url=RepoUrl('https://huggingface.co/nwiser/eng-darija-trsl', endpoint='https://huggingface.co', repo_type='model', repo_id='nwiser/eng-darija-trsl'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import pipeline

# Charger le modèle directement depuis le Hub
model_name = "nwiser/eng-darija-trsl"  # Remplacez par votre modèle
model_d_e = pipeline("text2text-generation", model=model_name)

# Tester le modèle
result = model_d_e("بغيت نشرب قهوة")
print(result)  # Sortie : [{'label': 'POSITIVE', 'score': 0.9998}]

Device set to use cuda:0


[{'generated_text': 'I want to drink coffee'}]
